As transformaçoes abaixo são validas somente para criar indicadores que utilizam agrupamentos de Clase Processual e Orgão Julgador

## SETUP

In [ ]:
import pandas as pd
from collections import Counter
from pymongo import MongoClient

In [ ]:
import datetime
import statistics

### Cria tabela de indicadores

In [ ]:
!pip install pymysql

     |████████████████████████████████| 51kB 1.7MB/s 


In [ ]:
tabela_indicadores = [(1, "Tempo médio entre a distribuição e o primeiro impulso do processo", "26 - Distribuição",	"11010 - Mero Expediente, 3 - Decisão, 11009 - Despacho, 1 - Magistrado, 1013 - Determinação, 12164 - Outras Decisões, 11383 - Ato Ordinatório, 12265 - Expedida/certificada", 	"1 vez", ""),
(2, "Tempo médio entre a distribuição e o envio para concluso",	"26 - Distribuição",	"51 - Conclusão",	"1 vez",	"Considera a primeira movimentação de distribuição e a movimentação de conclusão subsequente."),
(3, "Tempo médio entre a distribuição e a primeira audiência",	"26 - Distribuição",	"970 - Audiência",	"1 vez", ""),
(4, "Tempo médio entre a distribuição e a primeira ação do magistrado",	"26 - Distribuição",	"11010 - Mero Expediente, 3 - Decisão, 11009 - Despacho, 1 - Magistrado, 1013 - Determinação, 12164 - Outras Decisões",	"1 vez", "")]

In [ ]:
from pandas.io import sql
from sqlalchemy import create_engine

In [ ]:
# cria engine para usar no pandas
engine = create_engine('mysql+pymysql://root@35.238.86.130/HACKATHON_CNJ')

In [ ]:
df_indicadores = pd.DataFrame(tabela_indicadores, columns=["cod", "indicador", "condicao_1", "condicao_2", "unico_iterativo", "observacao"])

In [ ]:
df_indicadores.head()

,cod,indicador,condicao_1,condicao_2,unico_iterativo,observacao
0,1,Tempo médio entre a distribuição e o primeiro ...,26 - Distribuição,"11010 - Mero Expediente, 3 - Decisão, 11009 - ...",1 vez,
1,2,Tempo médio entre a distribuição e o envio par...,26 - Distribuição,51 - Conclusão,1 vez,Considera a primeira movimentação de distribui...
2,3,Tempo médio entre a distribuição e a primeira ...,26 - Distribuição,970 - Audiência,1 vez,
3,4,Tempo médio entre a distribuição e a primeira ...,26 - Distribuição,"11010 - Mero Expediente, 3 - Decisão, 11009 - ...",1 vez,


In [ ]:
# EXPORTA O DATAFRAME PARA UMA TABELA NO MYSQL

df_indicadores.to_sql(con=engine, name='indicadores', if_exists='replace')

## Conexão com o Mongo

In [ ]:
client = MongoClient('mongodb://35.237.232.73:27017/')

In [ ]:
db = client["justica_estadual"]

In [ ]:
coll = "processos-tjsp"

## Processo de agrupamento

In [ ]:
def get_process(db, collection, query={}):
  return list(db[collection].find(query))

In [ ]:
def split_process(data):
  """
    Recebe uma lista de processos e retorna uma nova lista
    agrupando processos por Classe Processual e por Orgão julgador

    {
      classe_processual: {
        orgao_julgador: [
          processo
        ]
      }
    }

  """
  process_list = {}

  for process in data:
    classe_processual = process["dadosBasicos"]["classeProcessual"]
    orgao_julgador = process["dadosBasicos"]["orgaoJulgador"]["codigoOrgao"]

    # caso não exista a classe processual na lista
    if not process_list.get(classe_processual):
      # cria um novo dict com a classe processual como chave
      process_list[classe_processual] = {}

    # caso não exista o orgao julgador dentro do dict da classe processual
    if not process_list[classe_processual].get(orgao_julgador):
      # aqui cria uma lista para adicionar o processo
      process_list[classe_processual][orgao_julgador] = []

    process_list[classe_processual][orgao_julgador].append(process)
  
  return process_list

In [ ]:
def get_range_dates(data, condition_1, condition_2):
  """
    data: Lista de processos agrupados por classe_processual e orgao_julgador
    condition_1: lista de inteiros -> DE
    condition_2: lista de inteiros -> Até

    Captura datas dos movimentos da condition_1 até a condition_2
  """
  group_range_dates = {}

  # for nos processos agrupados por Classe Processual e por Orgão julgador
  for classe_processual in data:
    group_range_dates[classe_processual] = {}

    # for nos orgaos
    for orgao_processual in data[classe_processual]:
      group_range_dates[classe_processual][orgao_processual] = []

      # for nos processos
      for processo in data[classe_processual][orgao_processual]:

        res_condition_1 = None
        res_condition_2 = None

        # se existir movimentos no processo
        if processo.get("movimento"):
          for mov in processo["movimento"]:
            
            # condições de nomenclatura no codigo do movimento
            if mov.get("movimentoNacional"):
              cod_mov = mov["movimentoNacional"]["codigoNacional"]
            elif mov.get("movimentoLocal"):
              cod_mov = mov["movimentoLocal"]["codigoPaiNacional"]
            else:
              cod_mov = 0
            
            if (cod_mov in condition_1) and (res_condition_1 is None):
              res_condition_1 = mov["dataHora"]
            elif (cod_mov in condition_2) and (res_condition_2 is None):
              res_condition_2 = mov["dataHora"]

            if (res_condition_1 is not None) and (res_condition_2 is not None):
              group_range_dates[classe_processual][orgao_processual].append(
                  (res_condition_1, res_condition_2)
              )
              break

  return group_range_dates

In [ ]:
def calc_median_df(range_dates):
  cals_medians = []

  for classe_processual in range_dates:
    for orgao_julgador in range_dates[classe_processual]:
      calc_median = []

      for cond_ranges in range_dates[classe_processual][orgao_julgador]:
        condition_1 = datetime.datetime.strptime(cond_ranges[0], '%Y%m%d%H%M%S')
        condition_2 = datetime.datetime.strptime(cond_ranges[1], '%Y%m%d%H%M%S')

        result = condition_2 - condition_1

        calc_median.append(int(result.days))
      
      if calc_median:
        cals_medians.append(
            (
                classe_processual,
                orgao_julgador,
                int(statistics.median(calc_median)),
                len(calc_median)
            )
        )
  
  return pd.DataFrame(cals_medians, columns=["CLASSE", "ORGÃO", "MÉDIA EM DIAS", "QUANTIDADE PROCESSOS"])

## Extração de Indicadores

#### Resgata todos os processos da collection

In [ ]:
process_data = get_process(db, coll)

#### Agrupa por orgão e classe

In [ ]:
group_process = split_process(process_data)

#### 26 - Distribuição e 848 - Trânsito em julgado

In [ ]:
range_dates_26_848 = get_range_dates(group_process, [26], [848])

In [ ]:
df = calc_median_df(range_dates_26_848)

In [ ]:
df.head()

,CLASSE,ORGÃO,MÉDIA EM DIAS,QUANTIDADE PROCESSOS
0,1116,10770,1670,4
1,1116,9444,17099,3
2,1116,9363,23171,4
3,1116,9824,16912,1
4,1116,9644,3548,1


In [ ]:
df.to_csv("./indicadores_distribuicao_transito_julgado.csv")

#### 26 - Distribuição e Primeiro impulso do processo

In [ ]:
cond_1 = [26]
cond_2 = [
          11010, # - Mero Expediente
          3, # - Decisão
          11009, # - Despacho
          1, # - Magistrado
          1013, # - Determinação
          12164, # - Outras Decisões
          11383, # - Ato Ordinatório
          12265 # - Expedida/certificada
]

In [ ]:
range_dates_26_ns = get_range_dates(group_process, cond_1, cond_2)

In [ ]:
df = calc_median_df(range_dates_26_ns)

In [ ]:
df.head()

,CLASSE,ORGÃO,MÉDIA EM DIAS,QUANTIDADE PROCESSOS
0,1116,10770,400,17
1,1116,9444,16723,3
2,1116,9304,17,2
3,1116,10212,111,102
4,1116,10355,1,56


In [ ]:
df.to_csv("./indicador_distribuição_primeiro_impulso.csv")

In [ ]:
df.to_sql(con=engine, name='indicador_1', if_exists='replace')

#### Tempo médio entre a distribuição e o envio para concluso

In [ ]:
cond_1 = [26]
cond_2 = [
          51 # Conclusão
]

In [ ]:
range_dates_26_ns = get_range_dates(group_process, cond_1, cond_2)

In [ ]:
df = calc_median_df(range_dates_26_ns)

In [ ]:
df.head()

,CLASSE,ORGÃO,MÉDIA EM DIAS,QUANTIDADE PROCESSOS
0,1116,10770,1121,17
1,1116,9304,15,2
2,1116,10212,98,102
3,1116,9925,17237,3
4,1116,10355,1,56


In [ ]:
df.to_csv("indicador_distribuição_envio_para_concluso.csv")

In [ ]:
df.to_sql(con=engine, name='indicador_2', if_exists='replace')

#### Tempo médio entre a distribuição e a primeira audiência

In [ ]:
cond_1 = [26]
cond_2 = [
          970 # Conclusão
]

In [ ]:
range_dates_26_ns = get_range_dates(group_process, cond_1, cond_2)

In [ ]:
df = calc_median_df(range_dates_26_ns)

In [ ]:
df.head()

,CLASSE,ORGÃO,MÉDIA EM DIAS,QUANTIDADE PROCESSOS
0,1116,9428,26,5
1,159,17582,1394,2
2,159,10489,1992,1
3,159,18301,2737,1
4,159,10439,515,1


In [ ]:
df.to_csv("indicador_distribuição_primeira_audiencia.csv")

In [ ]:
df.to_sql(con=engine, name='indicador_3', if_exists='replace')

#### Tempo médio entre a distribuição e a primeira ação do magistrado

In [ ]:
cond_1 = [26]
cond_2 = [
          11010, # - Mero Expediente
          3, # - Decisão
          11009, # - Despacho
          1, # - Magistrado
          1013, # - Determinação
          12164 # - Outras Decisões
]

In [ ]:
range_dates_26_ns = get_range_dates(group_process, cond_1, cond_2)

In [ ]:
df = calc_median_df(range_dates_26_ns)

In [ ]:
df.head()

,CLASSE,ORGÃO,MÉDIA EM DIAS,QUANTIDADE PROCESSOS
0,1116,10770,400,17
1,1116,9444,16723,3
2,1116,9304,17,2
3,1116,10212,111,102
4,1116,10355,1,56


In [ ]:
df.to_csv("./indicador_distribuicao_primeira_acao_do_magistrado.csv")

In [ ]:
df.to_sql(con=engine, name='indicador_4', if_exists='replace')